In [ ]:

# !gcloud dataproc clusters list --region us-central1
# Install a particular version of `google-cloud-storage` because (oddly enough) 
# the  version on Colab and GCP is old. A dependency error below is okay.
# !pip install -q google-cloud-storage==1.43.0
# !pip install -q graphframes
import pyspark
import sys
from collections import Counter, OrderedDict, defaultdict
import itertools
from itertools import islice, count, groupby
import pandas as pd
import os
import re
from operator import itemgetter
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
from time import time
from pathlib import Path
import pickle
import pandas as pd
from google.cloud import storage
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf, SparkFiles
from pyspark.sql import SQLContext
from graphframes import *
# adding our python module to the cluster
# sc.addFile("/home/dataproc/inverted_index_gcp.py")
# sys.path.insert(0,SparkFiles.getRootDirectory())

In [ ]:
# -*- coding: utf-8 -*-
"""Search_Engine.ipynb

Automatically generated by Colaboratory.

Original file is located at
https://colab.research.google.com/drive/1mrdj_nSJjTsjf2rImUddPzBymbvdznmx
"""

# These will already be installed in the testing environment so disregard the
# amount of time (~1 minute) it takes to install.
import subprocess

# if the following command generates an error, you probably didn't enable
# the cluster security option "Allow API access to all Google Cloud services"
# under Manage Security → Project Access when setting up the cluster
# subprocess.run(['pip', 'install', '-U', '-q', 'PyDrive'])
# subprocess.run(['apt', 'install', 'openjdk-8-jdk-headless', '-qq'])
# subprocess.run(['pip', 'install', '-q', 'graphframes'])
# subprocess.run(['pip', 'install', '-q', 'pyspark'])
# subprocess.run(['pip', 'install', '-q', 'gcsfs'])

# !pip install -U -q PyDrive
# !pip install -q gcsfs


# imports
import sys
from collections import Counter, OrderedDict
import itertools
from itertools import islice, count, groupby
import pandas as pd
import os
import re
from operator import itemgetter
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
from time import time
from timeit import timeit
from pathlib import Path
import pickle
import pandas as pd
import numpy as np
from google.cloud import storage
import pyspark
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext
from pyspark.ml.feature import Tokenizer, RegexTokenizer
from graphframes import *
import signal
import gcsfs

import hashlib
import inverted_index_gcp

nltk.download('stopwords')

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
# graphframes_jar = 'https://repos.spark-packages.org/graphframes/graphframes/0.8.2-spark3.2-s_2.12/graphframes-0.8.2-spark3.2-s_2.12.jar'
# spark_jars = '/usr/local/lib/python3.7/dist-packages/pyspark/jars'
# subprocess.run(['wget', '-N', '-P', spark_jars, graphframes_jar])

# client = storage.Client()

In [ ]:
## RENAME the project_id to yours project id from the project you created in GCP
# Roy: custom-octagon-370512
# Dudi: ir3-dudi-nassi
# project_id = 'custom-octagon-370512'
# subprocess.run(['gcloud', 'config', 'set', 'project', project_id])

# data_bucket_name = 'wikidata20210801_preprocessed'
# try:
#     if os.environ["wikidata_preprocessed"] is not None:
#         pass
# except:
#     subprocess.run(['mkdir', 'wikidumps'])
#     subprocess.run(['gsutil', '-u', project_id, 'cp', f'gs://{data_bucket_name}/*', 'wikidumps/'])

# try:
#     if os.environ["wikidata_preprocessed"] is not None:
#         path = os.environ["wikidata_preprocessed"]+"/wikidumps/*"
# except:
#     path = "wikidumps/*"

bucket_name = 'xxxxxx'
full_path = f"gs://{bucket_name}/"
paths=[]

client = storage.Client()
blobs = client.list_blobs(bucket_name)
for b in blobs:
    if b.name != 'graphframes.sh':
        paths.append(full_path+b.name)

In [ ]:
parquetFile = spark.read.parquet(*paths)


In [ ]:
# parquetFile.count()

In [ ]:
doc_title_pairs = parquetFile.select("title", "id").rdd
doc_text_pairs = parquetFile.select("text", "id").rdd
pages_links = parquetFile.select("id", "anchor_text").rdd

In [2]:
def _hash(s):
    return hashlib.blake2b(bytes(s, encoding='utf8'), digest_size=5).hexdigest()


NUM_BUCKETS = 124
def token2bucket_id(token):
    return int(_hash(token),16) % NUM_BUCKETS


def partition_postings_and_write(postings,bucket_name):
    # YOUR CODE HERE
    def to_list(a):
        return [a]

    def append(a, b):
        a.append(b)
        return a

    def extend(a, b):
        a.extend(b)
        return a
    bucket_id_rdd = postings.map(lambda x: (token2bucket_id(x[0]), (x[0], x[1])))
    write_list_rdd = bucket_id_rdd.combineByKey(to_list, append, extend).map(
        lambda x: inverted_index_gcp.InvertedIndex.write_a_posting_list(x, bucket_name))
    return write_list_rdd


from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from collections import defaultdict, Counter
import re
import nltk
import pickle
import numpy as np

nltk.download('stopwords')

from nltk.corpus import stopwords
from tqdm import tqdm
import operator
from itertools import islice, count
from contextlib import closing

import json
from io import StringIO
from pathlib import Path
from operator import itemgetter
import pickle
import matplotlib.pyplot as plt
import builtins


# TUPLE_SIZE = 6  # We're going to pack the doc_id and tf values in this
# # many bytes.
# TF_MASK = 2 ** 16 - 1  # Masking the 16 low bits of an integer

# DL = {}  # We're going to update and calculate this after each document. This will be usefull for the calculation of AVGDL (utilized in BM25)


from logging import exception

RE_WORD = re.compile(r"""[\#\@\w](['\-]?\w){2,24}""", re.UNICODE)
stopwords_frozen = frozenset(stopwords.words('english'))

corpus_stopwords = ["category", "references", "also", "external", "links", 
                    "may", "first", "see", "history", "people", "one", "two", 
                    "part", "thumb", "including", "second", "following", 
                    "many", "however", "would", "became"]

all_stopwords = stopwords_frozen.union(corpus_stopwords)


def tokenize(text):
    """
    This function aims in tokenize a text into a list of tokens. Moreover, it filter stopwords.

    Parameters:
    -----------
    text: string , represting the text to tokenize.

    Returns:
    -----------
    list of tokens (e.g., list of tokens).
    """
    list_of_tokens = [token.group() for token in RE_WORD.finditer(text.lower()) if
                      token.group() not in all_stopwords]
    return list_of_tokens


def generate_query_tfidf_vector(query_to_search, index):
    """
    Generate a vector representing the query. Each entry within this vector represents a tfidf score.
    The terms representing the query will be the unique terms in the index.

    We will use tfidf on the query as well.
    For calculation of IDF, use log with base 10.
    tf will be normalized based on the length of the query.

    Parameters:
    -----------
    query_to_search: list of tokens (str). This list will be preprocessed in advance (e.g., lower case, filtering stopwords, etc.').
                     Example: 'Hello, I love information retrival' --->  ['hello','love','information','retrieval']

    index:           inverted index loaded from the corresponding files.

    Returns:
    -----------
    vectorized query with tfidf scores
    """

    epsilon = .0000001
    total_vocab_size = len(index.term_total)
    Q = np.zeros((total_vocab_size))
    term_vector = list(index.term_total.keys())
    counter = Counter(query_to_search)
    for token in np.unique(query_to_search):
        if token in index.term_total.keys():  # avoid terms that do not appear in the index.
            tf = counter[token] / len(query_to_search)  # term frequency divded by the length of the query
            df = index.df[token]
            idf = math.log((len(index.DL)) / (df + epsilon), 10)  # smoothing

            try:
                ind = term_vector.index(token)
                Q[ind] = tf * idf
            except:
                pass
    return Q


def get_posting_iter(index):
    """
    This function returning the iterator working with posting list.

    Parameters:
    ----------
    index: inverted index
    """
    words, pls = zip(*index.posting_lists_iter())
    return words, pls


def get_candidate_documents_and_scores(query_to_search, index, words, pls):
    """
    Generate a dictionary representing a pool of candidate documents for a given query. This function will go through every token in query_to_search
    and fetch the corresponding information (e.g., term frequency, document frequency, etc.') needed to calculate TF-IDF from the posting list.
    Then it will populate the dictionary 'candidates.'
    For calculation of IDF, use log with base 10.
    tf will be normalized based on the length of the document.

    Parameters:
    -----------
    query_to_search: list of tokens (str). This list will be preprocessed in advance (e.g., lower case, filtering stopwords, etc.').
                     Example: 'Hello, I love information retrival' --->  ['hello','love','information','retrieval']

    index:           inverted index loaded from the corresponding files.

    words,pls: iterator for working with posting.

    Returns:
    -----------
    dictionary of candidates. In the following format:
                                                               key: pair (doc_id,term)
                                                               value: tfidf score.
    """
    candidates = {}
    for term in np.unique(query_to_search):
        if term in words:
            list_of_doc = pls[words.index(term)]
            normlized_tfidf = [(doc_id, (freq / index.DL[doc_id]) * math.log(len(index.DL) / index.df[term], 10)) for doc_id, freq
                               in list_of_doc]

            for doc_id, tfidf in normlized_tfidf:
                candidates[(doc_id, term)] = candidates.get((doc_id, term), 0) + tfidf

    return candidates


def generate_document_tfidf_matrix(query_to_search, index, words, pls):
    """
    Generate a DataFrame `D` of tfidf scores for a given query.
    Rows will be the documents candidates for a given query
    Columns will be the unique terms in the index.
    The value for a given document and term will be its tfidf score.

    Parameters:
    -----------
    query_to_search: list of tokens (str). This list will be preprocessed in advance (e.g., lower case, filtering stopwords, etc.').
                     Example: 'Hello, I love information retrival' --->  ['hello','love','information','retrieval']

    index:           inverted index loaded from the corresponding files.


    words,pls: iterator for working with posting.

    Returns:
    -----------
    DataFrame of tfidf scores.
    """

    total_vocab_size = len(index.term_total)
    candidates_scores = get_candidate_documents_and_scores(query_to_search, index, words,
                                                           pls)  # We do not need to utilize all document. Only the docuemnts which have corrspoinding terms with the query.
    unique_candidates = np.unique([doc_id for doc_id, freq in candidates_scores.keys()])
    D = np.zeros((len(unique_candidates), total_vocab_size))
    D = pd.DataFrame(D)

    D.index = unique_candidates
    D.columns = index.term_total.keys()

    for key in candidates_scores:
        tfidf = candidates_scores[key]
        doc_id, term = key
        D.loc[doc_id][term] = tfidf

    return D


import numpy as np
from numpy.linalg import norm


# def cosine_similarity(D, Q):
#     """
#     Calculate the cosine similarity for each candidate document in D and a given query (e.g., Q).
#     Generate a dictionary of cosine similarity scores
#     key: doc_id
#     value: cosine similarity score

#     Parameters:
#     -----------
#     D: DataFrame of tfidf scores.

#     Q: vectorized query with tfidf scores

#     Returns:
#     -----------
#     dictionary of cosine similarity score as follows:
#                                                                 key: document id (e.g., doc_id)
#                                                                 value: cosine similarty score.
#     """
#     # YOUR CODE HERE
#     d = {}
#     for index, doc in D.iterrows():
#         m = np.dot(doc, Q) / (norm(doc) * norm(Q))
#         d[index] = m
#     return d


# from sklearn.metrics.pairwise import cosine_similarity
# from scipy import sparse
# def fast_cosine_similarity(D, Q):
#   d=np.array(D)
#   q=np.array(Q)
#   d_sparse=sparse.csr_matrix(d)
#   q_sparse=sparse.csr_matrix(q)
#   return cosine_similarity(d_sparse,q_sparse)


# Put your `generate_graph` function here
def generate_graph(pages):
  # YOUR CODE HERE
  edges = pages.flatMapValues(lambda x: x).map(lambda x: (x[0],x[1][0])).distinct()
  vertices1 = edges.values()
  vertices2 = edges.keys()
  vertices = vertices1.union(vertices2).distinct().map(lambda x: (x,x))
  return edges, vertices


def get_top_n(sim_dict, N=3):
    """
    Sort and return the highest N documents according to the cosine similarity score.
    Generate a dictionary of cosine similarity scores

    Parameters:
    -----------
    sim_dict: a dictionary of similarity score as follows:
                                                                key: document id (e.g., doc_id)
                                                                value: similarity score. We keep up to 5 digits after the decimal point. (e.g., round(score,5))

    N: Integer (how many documents to retrieve). By default N = 3

    Returns:
    -----------
    a ranked list of pairs (doc_id, score) in the length of N.
    """

    return builtins.sorted([(doc_id, builtins.round(score, 5)) for doc_id, score in sim_dict.items()],
                           key=lambda x: x[1], reverse=True)


def get_topN_score_for_queries(queries_to_search, index, N=3):
    """
    Generate a dictionary that gathers for every query its topN score.

    Parameters:
    -----------
    queries_to_search: a dictionary of queries as follows:
                                                        key: query_id
                                                        value: list of tokens.
    index:           inverted index loaded from the corresponding files.
    N: Integer. How many documents to retrieve. This argument is passed to the topN function. By default N = 3, for the topN function.

    Returns:
    -----------
    return: a dictionary of queries and topN pairs as follows:
                                                        key: query_id
                                                        value: list of pairs in the following format:(doc_id, score).
    """
    # YOUR CODE HERE
    dict1 = {}
    d = get_posting_iter(index)
    i = 0
    for key, query in queries_to_search.items():
        i += 1
        Q = generate_query_tfidf_vector(query, index)
        D = generate_document_tfidf_matrix(query, index, d[0], d[1])
        x = fast_cosine_similarity(D, Q)
        z = get_top_n(x, N)
        dict1[key] = z

    return dict1


import math
from itertools import chain
import time


# When preprocessing the data have a dictionary of document length for each document saved in a variable called `DL`.
class BM25_from_index:
    """
    Best Match 25.
    ----------
    k1 : float, default 1.5

    b : float, default 0.75

    index: inverted index
    """

    def __init__(self, index, k1=1.5, b=0.75):
        self.b = b
        self.k1 = k1
        self.index = index
        self.N = len(index.DL)
        self.AVGDL = builtins.sum(index.DL.values()) / self.N
        self.words, self.pls = zip(*self.index.posting_lists_iter())

    def calc_idf(self, list_of_tokens):
        """
        This function calculate the idf values according to the BM25 idf formula for each term in the query.

        Parameters:
        -----------
        query: list of token representing the query. For example: ['look', 'blue', 'sky']

        Returns:
        -----------
        idf: dictionary of idf scores. As follows:
                                                    key: term
                                                    value: bm25 idf score
        """
        idf = {}
        for term in list_of_tokens:
            if term in self.index.df.keys():
                n_ti = self.index.df[term]
                idf[term] = math.log(1 + (self.N - n_ti + 0.5) / (n_ti + 0.5))
            else:
                pass
        return idf

    def search(self, queries, N=3):
        """
        This function calculate the bm25 score for given query and document.
        We need to check only documents which are 'candidates' for a given query.
        This function return a dictionary of scores as the following:
                                                                    key: query_id
                                                                    value: a ranked list of pairs (doc_id, score) in the length of N.

        Parameters:
        -----------
        query: list of token representing the query. For example: ['look', 'blue', 'sky']
        doc_id: integer, document id.

        Returns:
        -----------
        score: float, bm25 score.
        """
        # YOUR CODE HERE
        res = {}
        # words, pls = get_posting_iter(self.index)
        for query_id in queries:
            self.idf = self.calc_idf(queries[query_id])
            c = get_candidate_documents_and_scores(queries[query_id], self.index, self.words, self.pls)
            cd = np.unique([doc[0] for doc in c.keys()])
            sim = {doc: self._score(queries[query_id], doc) for doc in cd}
            res[query_id] = get_top_n(sim, N)
        return res

    def _score(self, query, doc_id):
        """
        This function calculate the bm25 score for given query and document.

        Parameters:
        -----------
        query: list of token representing the query. For example: ['look', 'blue', 'sky']
        doc_id: integer, document id.

        Returns:
        -----------
        score: float, bm25 score.
        """
        score = 0.0
        doc_len = self.index.DL[doc_id]
        for term in query:
            if term in self.index.term_total.keys():
                term_frequencies = dict(self.pls[self.words.index(term)])
                if doc_id in term_frequencies.keys():
                    freq = term_frequencies[doc_id]
                    numerator = self.idf[term] * freq * (self.k1 + 1)
                    denominator = freq + self.k1 * (1 - self.b + self.b * doc_len / self.AVGDL)
                    score += (numerator / denominator)
        return score
        score = 0
        # tune k1
        # if len(query) > self.avgdl_:
        #   K1 = self.k1 + 0.5
        # elif len(query) == self.avgdl_:
        #   K1 = self.k1 + 0.1
        # else:
        #   K1 = self.k1 + 0.3
        K1 = self.k1
        # tune b
        doc_len_ = self.index.DL[doc_id]

        B = 1 - self.b + self.b * (doc_len_/self.AVGDL)
        # remove word that not in the corpus
        for t in list(query):
          if t not in self.df_.keys():
            query.remove(t)
        # calculate idf
        idf_score = self.calc_idf(query)

        #  calculate score
        for t in query:
          if self.tf_[doc_id][t] > 0:
            tf_ij = ((K1 + 1) * self.tf_[doc_id][t]) / (B * K1 + self.tf_[doc_id][t])
            score += tf_ij * idf_score[t]
        return score






# def merge_results(title_scores, body_scores, title_weight=0.5, text_weight=0.5, N=3):
#     """
#     This function merge and sort documents retrieved by its weighte score (e.g., title and body).

#     Parameters:
#     -----------
#     title_scores: a dictionary build upon the title index of queries and tuples representing scores as follows:
#                                                                             key: query_id
#                                                                             value: list of pairs in the following format:(doc_id,score)

#     body_scores: a dictionary build upon the body/text index of queries and tuples representing scores as follows:
#                                                                             key: query_id
#                                                                             value: list of pairs in the following format:(doc_id,score)
#     title_weight: float, for weigted average utilizing title and body scores
#     text_weight: float, for weigted average utilizing title and body scores
#     N: Integer. How many document to retrieve. This argument is passed to topN function. By default N = 3, for the topN function.

#     Returns:
#     -----------
#     dictionary of querires and topN pairs as follows:
#                                                         key: query_id
#                                                         value: list of pairs in the following format:(doc_id,score).
#     """

#     # result: dict[query] = [(doc_id, title_score*weight + body_score*weight), (doc_id …)...]
#     # YOUR CODE HERE
#     def get_top_n_without_round(sim_dict, N=3):
#         return sorted([(doc_id, score) for doc_id, score in sim_dict.items()], key=lambda x: x[1], reverse=True)

#     weighted_dict = {query_id: {tup[0]: tup[1] * title_weight for tup in title_scores[query_id]} for query_id in
#                      title_scores}

#     for query_id in body_scores:
#         for tup in body_scores[query_id]:
#             doc_id = tup[0]
#             text_weighted = tup[1] * text_weight
#             if doc_id not in weighted_dict[query_id].keys():
#                 weighted_dict[query_id][doc_id] = text_weighted
#             else:
#                 weighted_dict[query_id][doc_id] += text_weighted
#         weighted_dict[query_id] = get_top_n_without_round(weighted_dict[query_id], N)
#     return weighted_dict


def calculate_df(postings):
  # YOUR CODE HERE
  result = postings.map(lambda x: (x[0],len(x[1])))
  return result


def get_id_name_list(half_and_half, doc_title_pairs_dict):
    return [(str(tup[0]), doc_title_pairs_dict[tup[0]]) for query_id in half_and_half for tup in
            half_and_half[query_id]]


def tokenize_bin(text):
    """
    This function aims in tokenize a text into a list of tokens. Moreover, it filter stopwords.

    Parameters:
    -----------
    text: string , represting the text to tokenize.

    Returns:
    -----------
    list of tokens (e.g., list of tokens).
    """
    list_of_tokens = set([token.group() for token in RE_WORD.finditer(text.lower()) if
                      token.group() not in all_stopwords])
    return list(list_of_tokens)


def reduce_word_counts(unsorted_pl):
    lst = sorted(unsorted_pl, key=lambda x:x[0])
    return lst


def word_count(text, id):
  tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
  # YOUR CODE HERE
  tokens_without_stopword = [word for word in tokens if word not in all_stopwords]
  result_dict = {}
  if len(tokens_without_stopword) == 0:
    tokens_without_stopword = tokens
  for word in tokens_without_stopword:
    if word not in result_dict:
      result_dict[word] = [id,1]
    else:
      result_dict[word][1] += 1
  map_reduce_res = [(key,tuple(result_dict[key])) for key in result_dict]
  return map_reduce_res


def word_count_bin(text, id):
    tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
    lst_words = []
    cnt = Counter(i for i in tokens if i not in all_stopwords)
    if len(cnt) == 0:
      cnt = Counter(i for i in tokens)
    for i in cnt.keys():
        lst_words.append((i,(id,1)))
    return lst_words


# Getting tokens from the text while removing punctuations.
def stemmer_tokens(tokens):
  # YOUR CODE HERE
  stemmer = PorterStemmer()
  tokens = tokens.split(" ")
  tokens_removed = [word for word in tokens if word not in all_stopwords]
  if len(tokens_removed) == 0:
    tokens_removed = tokens
  tokens_stemmed = [stemmer.stem(word) for word in tokens_removed]
  tokens_stemmed = ' '.join(tokens_stemmed)
  return tokens_stemmed


def fast_super_duper_cosine(query,index,words,pls):
  scores = {}
  for term in query:
    list_of_candidates = []
    if term in words:
        list_of_doc = pls[words.index(term)]
        normlized_tfidf = [(doc_id, (freq / index.DL[(doc_id)]) * math.log(len(index.DL) / index.df[term], 10)) for
                            doc_id, freq in list_of_doc]
        for i in range(len(list_of_doc)):
          if list_of_doc[i][0] not in scores.keys():
            scores[list_of_doc[i][0]] = normlized_tfidf[i][1] * list_of_doc[i][1]
          else:
            scores[list_of_doc[i][0]] += normlized_tfidf[i][1] * list_of_doc[i][1]
  for doc_id in scores.keys():
    scores[doc_id] = scores[doc_id]/index.DL[doc_id]
  return sorted(scores.items(),key=lambda x: x[1],reverse=True)[:100]


def merge_results(title_scores,body_scores,anchor_scores,title_weight=0.6,text_weight=0.3,anchor_weight=0.1, N=3):    
    """
    This function merge and sort documents retrieved by its weighte score (e.g., title and body). 

    Parameters:
    -----------
    title_scores: a dictionary build upon the title index of queries and tuples representing scores as follows: 
                                                                            key: query_id
                                                                            value: list of pairs in the following format:(doc_id,score)
                
    body_scores: a dictionary build upon the body/text index of queries and tuples representing scores as follows: 
                                                                            key: query_id
                                                                            value: list of pairs in the following format:(doc_id,score)
    title_weight: float, for weigted average utilizing title and body scores
    text_weight: float, for weigted average utilizing title and body scores
    N: Integer. How many document to retrieve. This argument is passed to topN function. By default N = 3, for the topN function. 
    
    Returns:
    -----------
    dictionary of querires and topN pairs as follows:
                                                        key: query_id
                                                        value: list of pairs in the following format:(doc_id,score). 
    """
    # YOUR CODE HERE
    def get_top_n_new(sim_dict,N):
      return sorted([(doc_id,score) for doc_id, score in sim_dict.items()], key = lambda x: x[1],reverse=True)

    d={}
    res={}
    for query_id,value in title_scores.items():
      lst1=[(x[0],title_weight * x[1]) for x in value]
      for doc_id,score in lst1:
        d[doc_id]=score 
      value2=body_scores[query_id]
      value3=anchor_scores[query_id]
      lst2=[(x[0],text_weight * x[1]) for x in value2 ]
      lst3=[(x[0],anchor_weight * x[1]) for x in value3]
      for doc_id,score in lst2:
          if(doc_id in d):
            d[doc_id]+=score
          else:
            d[doc_id]=score
      for doc_id,score in lst3:
          if(doc_id in d):
            d[doc_id]+=score
          else:
            d[doc_id]=score      
      x=get_top_n_new(d,N)
      res[query_id]=x
      d={} 
    return res


def search_binary(query,index,words,pls):
  scores = {}
  for term in query:
    if term in words:
        list_of_doc = pls[words.index(term)]
        for doc_id, freq in list_of_doc:
          if doc_id not in scores:
            scores[doc_id] = 1
          else:
            scores[doc_id] += 1
  return sorted(scores.items(),key=lambda x:x[1],reverse=True)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
def write_data_stemmed(name, doc_pairs, title_pairs):
    # title
    if name == 'title':
        # bucket for title
        bucket_name = 'title_bucket_roy_dudi_stemmed'
        # convert rdd to list
        doc_title_pairs_ = doc_pairs.collect()
        # stemm
        doc_pairs = doc_pairs.map(lambda x :(x[1],stemmer_tokens(x[0])))
        # convert list to dict{id:text}
        doc_title_pairs_dict = {element[1]: element[0] for element in doc_title_pairs_}
        # tokenize the dicts
        # doc_data_titles = {k: tokenize_bin(v) for k, v in doc_title_pairs_dict.items() if len(tokenize_bin(v)) != 0}        
        # get posting list
        word_counts = doc_pairs.flatMap(lambda x: word_count_bin(x[1], x[0]))
        postings = word_counts.groupByKey().mapValues(reduce_word_counts)
        # partition posting lists and write out
        _ = partition_postings_and_write(postings, bucket_name).collect()
        # collect all posting lists locations into one super-set
        super_posting_locs = defaultdict(list)
        for blob in client.list_blobs(bucket_name, prefix='postings_gcp'):
            if not blob.name.endswith("pickle"):
                continue
            with blob.open("rb") as f:
                posting_locs = pickle.load(f)
                for k, v in posting_locs.items():
                    super_posting_locs[k].extend(v)
        # create inverted index
        inverted_index_titles = inverted_index_gcp.InvertedIndex(doc_pairs_dict=doc_title_pairs_dict, bucket_name=bucket_name)        
        inverted_index_titles.posting_locs = super_posting_locs
        w2df = calculate_df(postings)
        w2df_dict = w2df.collectAsMap()
        inverted_index_titles.df = w2df_dict
        for w, pl in inverted_index_titles.posting_lists_iter():
            result = 0
            for posting in pl:
                if posting[0] not in inverted_index_titles.DL:
                    inverted_index_titles.DL[posting[0]] = posting[1]
                else:
                    inverted_index_titles.DL[posting[0]] += posting[1]
                result += posting[1]
            inverted_index_titles.term_total[w] = result
        # write to memory each inverted index
        inverted_index_titles.write_index('.', "title_index")
        index_src = "title_index.pkl"
        index_dst = f'gs://{bucket_name}/title_index/{index_src}'
        subprocess.run(['gsutil', 'cp', index_src, index_dst])

    # body
    if name == 'body':
        # bucket for body
        bucket_name = 'body_bucket_roy_dudi_stemmed'
        # use title for names returns
        # stemm
        doc_pairs = doc_pairs.map(lambda x :(x[1],stemmer_tokens(x[0])))
        doc_title_pairs_ = title_pairs.collect()
        doc_title_pairs_dict = {element[1]: element[0] for element in doc_title_pairs_}
        # get posting list
        word_counts = doc_pairs.flatMap(lambda x: word_count(x[1], x[0]))
        postings = word_counts.groupByKey().mapValues(reduce_word_counts)
        postings_filtered = postings.filter(lambda x: len(x[1])>50)
        # partition posting lists and write out
        _ = partition_postings_and_write(postings_filtered, bucket_name).collect()
        # collect all posting lists locations into one super-set
        super_posting_locs = defaultdict(list)
        for blob in client.list_blobs(bucket_name, prefix='postings_gcp'):
            if not blob.name.endswith("pickle"):
                continue
            with blob.open("rb") as f:
                posting_locs = pickle.load(f)
                for k, v in posting_locs.items():
                    super_posting_locs[k].extend(v)
        # create inverted index
        inverted_index_text = inverted_index_gcp.InvertedIndex(doc_pairs_dict=doc_title_pairs_dict, bucket_name=bucket_name)
        w2df = calculate_df(postings_filtered)
        w2df_dict = w2df.collectAsMap()
        inverted_index_text.df = w2df_dict
        inverted_index_text.posting_locs = super_posting_locs
        for w, pl in inverted_index_text.posting_lists_iter():
            result = 0
            for posting in pl:
                if posting[0] not in inverted_index_text.DL:
                    inverted_index_text.DL[posting[0]] = posting[1]
                else:
                    inverted_index_text.DL[posting[0]] += posting[1]
                result += posting[1]
            inverted_index_text.term_total[w] = result
        # write to memory each inverted index
        inverted_index_text.write_index('.', "body_index")
        index_src = "body_index.pkl"
        index_dst = f'gs://{bucket_name}/body_index/{index_src}'
        subprocess.run(['gsutil', 'cp', index_src, index_dst])


    # anchor
    if name == 'anchor':
        # bucket for anchor
        bucket_name = 'anchor_bucket_roy_dudi_stemmed'
        # use title for names returs
        # stemm
        doc_pairs = doc_pairs.map(lambda x :(x[0],stemmer_tokens(x[1])))
        doc_title_pairs_ = title_pairs.collect()
        doc_title_pairs_dict = {element[1]: element[0] for element in doc_title_pairs_}
        # get posting list
        pages_links_flat = pages_links.flatMap(lambda x: x[1])
        word_counts = pages_links_flat.flatMap(lambda x: word_count_bin(x[1], x[0]))
        postings = word_counts.groupByKey().mapValues(reduce_word_counts)
        # partition posting lists and write out
        _ = partition_postings_and_write(postings, bucket_name).collect()
        # collect all posting lists locations into one super-set
        super_posting_locs = defaultdict(list)
        for blob in client.list_blobs(bucket_name, prefix='postings_gcp'):
            if not blob.name.endswith("pickle"):
                continue
            with blob.open("rb") as f:
                posting_locs = pickle.load(f)
                for k, v in posting_locs.items():
                    super_posting_locs[k].extend(v)
        # create inverted index
        inverted_index_anchor = inverted_index_gcp.InvertedIndex(doc_pairs_dict=doc_title_pairs_dict, bucket_name=bucket_name)
        inverted_index_anchor.posting_locs = super_posting_locs
        w2df = calculate_df(postings)
        w2df_dict = w2df.collectAsMap()
        inverted_index_anchor.df = w2df_dict
        for w, pl in inverted_index_anchor.posting_lists_iter():
            result = 0
            for posting in pl:
                if posting[0] not in inverted_index_anchor.DL:
                    inverted_index_anchor.DL[posting[0]] = posting[1]
                else:
                    inverted_index_anchor.DL[posting[0]] += posting[1]
                result += posting[1]
            inverted_index_anchor.term_total[w] = result
        # write to memory each inverted index
        inverted_index_anchor.write_index('.', "anchor_index")
        index_src = "anchor_index.pkl"
        index_dst = f'gs://{bucket_name}/anchor_index/{index_src}'
        subprocess.run(['gsutil', 'cp', index_src, index_dst])




In [ ]:
write_data_stemmed("title", doc_title_pairs, doc_title_pairs)
write_data_stemmed("body", doc_text_pairs, doc_title_pairs)
write_data_stemmed("anchor", pages_links, doc_title_pairs)


In [ ]:
def write_data(name, doc_pairs, title_pairs):
    # title
    if name == 'title':
        # bucket for title
        bucket_name = 'title_bucket_roy_dudi'
        # convert rdd to list
        doc_title_pairs_ = doc_pairs.collect()
        # convert list to dict{id:text}
        doc_title_pairs_dict = {element[1]: element[0] for element in doc_title_pairs_}
        # tokenize the dicts
        # doc_data_titles = {k: tokenize_bin(v) for k, v in doc_title_pairs_dict.items() if len(tokenize_bin(v)) != 0}        
        # get posting list
        word_counts = doc_pairs.flatMap(lambda x: word_count_bin(x[0], x[1]))
        postings = word_counts.groupByKey().mapValues(reduce_word_counts)
        # partition posting lists and write out
        _ = partition_postings_and_write(postings, bucket_name).collect()
        # collect all posting lists locations into one super-set
        super_posting_locs = defaultdict(list)
        for blob in client.list_blobs(bucket_name, prefix='postings_gcp'):
            if not blob.name.endswith("pickle"):
                continue
            with blob.open("rb") as f:
                posting_locs = pickle.load(f)
                for k, v in posting_locs.items():
                    super_posting_locs[k].extend(v)
        # create inverted index
        inverted_index_titles = inverted_index_gcp.InvertedIndex(doc_pairs_dict=doc_title_pairs_dict, bucket_name=bucket_name)        
        inverted_index_titles.posting_locs = super_posting_locs
        w2df = calculate_df(postings)
        w2df_dict = w2df.collectAsMap()
        inverted_index_titles.df = w2df_dict
        for w, pl in inverted_index_titles.posting_lists_iter():
            result = 0
            for posting in pl:
                if posting[0] not in inverted_index_titles.DL:
                    inverted_index_titles.DL[posting[0]] = posting[1]
                else:
                    inverted_index_titles.DL[posting[0]] += posting[1]
                result += posting[1]
            inverted_index_titles.term_total[w] = result
        # write to memory each inverted index
        inverted_index_titles.write_index('.', "title_index")
        index_src = "title_index.pkl"
        index_dst = f'gs://{bucket_name}/title_index/{index_src}'
        subprocess.run(['gsutil', 'cp', index_src, index_dst])

    # body
    if name == 'body':
        # bucket for body
        bucket_name = 'body_bucket_roy_dudi'
        # use title for names returns
        doc_title_pairs_ = title_pairs.collect()
        doc_title_pairs_dict = {element[1]: element[0] for element in doc_title_pairs_}
        # get posting list
        word_counts = doc_pairs.flatMap(lambda x: word_count(x[0], x[1]))
        postings = word_counts.groupByKey().mapValues(reduce_word_counts)
        postings_filtered = postings.filter(lambda x: len(x[1])>50)
        # partition posting lists and write out
        _ = partition_postings_and_write(postings_filtered, bucket_name).collect()
        # collect all posting lists locations into one super-set
        super_posting_locs = defaultdict(list)
        for blob in client.list_blobs(bucket_name, prefix='postings_gcp'):
            if not blob.name.endswith("pickle"):
                continue
            with blob.open("rb") as f:
                posting_locs = pickle.load(f)
                for k, v in posting_locs.items():
                    super_posting_locs[k].extend(v)
        # create inverted index
        inverted_index_text = inverted_index_gcp.InvertedIndex(doc_pairs_dict=doc_title_pairs_dict, bucket_name=bucket_name)
        w2df = calculate_df(postings_filtered)
        w2df_dict = w2df.collectAsMap()
        inverted_index_text.df = w2df_dict
        inverted_index_text.posting_locs = super_posting_locs
        for w, pl in inverted_index_text.posting_lists_iter():
            result = 0
            for posting in pl:
                if posting[0] not in inverted_index_text.DL:
                    inverted_index_text.DL[posting[0]] = posting[1]
                else:
                    inverted_index_text.DL[posting[0]] += posting[1]
                result += posting[1]
            inverted_index_text.term_total[w] = result
        # write to memory each inverted index
        inverted_index_text.write_index('.', "body_index")
        index_src = "body_index.pkl"
        index_dst = f'gs://{bucket_name}/body_index/{index_src}'
        subprocess.run(['gsutil', 'cp', index_src, index_dst])



    # anchor
    if name == 'anchor':
        # bucket for anchor
        bucket_name = 'anchor_bucket_roy_dudi'
        # use title for names returs
        doc_title_pairs_ = title_pairs.collect()
        doc_title_pairs_dict = {element[1]: element[0] for element in doc_title_pairs_}
        # get posting list
        pages_links_flat = pages_links.flatMap(lambda x: x[1])
        word_counts = pages_links_flat.flatMap(lambda x: word_count_bin(x[1], x[0]))
        postings = word_counts.groupByKey().mapValues(reduce_word_counts)
        # partition posting lists and write out
        _ = partition_postings_and_write(postings, bucket_name).collect()
        # collect all posting lists locations into one super-set
        super_posting_locs = defaultdict(list)
        for blob in client.list_blobs(bucket_name, prefix='postings_gcp'):
            if not blob.name.endswith("pickle"):
                continue
            with blob.open("rb") as f:
                posting_locs = pickle.load(f)
                for k, v in posting_locs.items():
                    super_posting_locs[k].extend(v)
        # create inverted index
        inverted_index_anchor = inverted_index_gcp.InvertedIndex(doc_pairs_dict=doc_title_pairs_dict, bucket_name=bucket_name)
        inverted_index_anchor.posting_locs = super_posting_locs
        w2df = calculate_df(postings)
        w2df_dict = w2df.collectAsMap()
        inverted_index_anchor.df = w2df_dict
        for w, pl in inverted_index_anchor.posting_lists_iter():
            result = 0
            for posting in pl:
                if posting[0] not in inverted_index_anchor.DL:
                    inverted_index_anchor.DL[posting[0]] = posting[1]
                else:
                    inverted_index_anchor.DL[posting[0]] += posting[1]
                result += posting[1]
            inverted_index_anchor.term_total[w] = result
        # write to memory each inverted index
        inverted_index_anchor.write_index('.', "anchor_index")
        index_src = "anchor_index.pkl"
        index_dst = f'gs://{bucket_name}/anchor_index/{index_src}'
        subprocess.run(['gsutil', 'cp', index_src, index_dst])
